In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, LinearRegression
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

RSEED = 42

In [ ]:
df_train_prepro = pd.read_csv('data/preprocessed_train_data_with_date_hol_concat.csv')

In [3]:
df_train_prepro

,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,dep_hour,dep_day,...,dep_temp,dep_precip,dep_wind,arr_temp,arr_precip,arr_wind,Country,City,is_holiday,holiday_length
0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-19.0,9300.0,4,16,...,9.4,9.9,23.6,12.5,0.0,16.5,Tunisia,Tunis,0,0
1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-48.0,10200.0,14,17,...,11.7,0.0,41.2,2.7,2.6,8.8,Tunisia,Djerba,0,0
2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-16.0,15600.0,19,20,...,11.1,1.3,6.8,22.3,23.9,11.8,Tunisia,Tunis,0,0
3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-8.0,10200.0,20,21,...,5.2,0.0,10.8,10.3,0.0,5.0,Turkey,Istanbul,0,0
4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-37.0,8700.0,17,22,...,16.0,0.0,9.8,10.6,2.0,4.6,Morocco,Casablanca,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0,8,18,...,16.8,0.0,14.2,17.1,0.0,20.2,Tunisia,Tunis,0,0
99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0,10,5,...,9.6,1.2,10.7,17.3,0.0,12.5,France,Paris,0,0
99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0,9,5,...,19.1,0.0,12.4,17.3,0.0,12.5,Belgium,Brussels,0,0
99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0,18,4,...,10.2,1.2,8.2,16.7,0.0,19.6,France,Paris,0,0


In [19]:
aircraft_unique = df_train_prepro['aircraft_code'].unique()
aircraft_unique

array(['TU 32AIMN', 'TU 736IOK', 'TU 320IMR', 'TU 320IMU', 'TU 31AIMJ',
       'TU 31BIMQ', 'TU 31BIMO', 'TU 736IOR', 'TU 320IMT', 'TU 320IMW',
       'TU 320IMV', 'TU 32AIMF', 'TU 320IMS', 'TU 32AIMI', 'TU 32AIML',
       'TU 32AIMG', 'TU 736ION', 'TU 32AIMM', 'TU 32AIMH', 'TU 736IOL',
       'TU 31AIMK', 'TU 736IOQ', 'TU 32AIMD', 'TU 32AIMC', 'TU 32AIMP',
       'TU 736IOM', 'TU CR9ISA', 'TU 736IOP', 'TU 332IFM', '5M 343SUN',
       'TU 332IFN', '5K 343TQY', 'UG AT7LBD', 'UG AT7LBE', 'UG AT7AT7',
       'BJ 320INQ', 'BJ 320INP', 'BJ 320INA', 'UG CR9XXX', 'UG CR9CR9',
       'TU 32A32A', 'BJ 320INH', 'BJ 320INB', 'GJ 734CGC', 'PS 738PSD',
       'D4 319CQG', 'QS 738TSC', 'UJ 320TCF', 'BJ 320INC', '5M 343JAI',
       '5K 345TFX', 'D4 320CQP', 'UG CR9ISA', 'GW 320MQH', 'TU CR9CR9',
       '6P M87TRJ', 'TU CR9XXX', 'GJ 733GGC', 'GJ 733EWE', 'GJ 733LGC',
       'GJ 734PGC', 'GJ 734MGC', '5M 343FOX', 'BJ 320INR', 'BJ 320INO',
       'OL 321ABY', 'X9 320VEB', 'TU 320320'], dtype=object)

In [16]:
aircraft_unique.to_csv('data/aircraft_unique.csv', index=False)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [17]:
# Convert numpy array to pandas Series before saving to CSV
pd.Series(aircraft_unique).to_csv('data/aircraft_unique.csv', index=False)

In [20]:
df_aircraft_model = pd.read_csv('data/full_aircraft_model_mapping_CORRECTED.csv')

In [21]:
df_aircraft_model

,aircraft_code,aircraft_model
0,TU 32AIMN,Airbus A320
1,TU 736IOK,Boeing 737-600
2,TU 320IMR,Airbus A320-200
3,TU 320IMU,Airbus A320-200
4,TU 31AIMJ,Airbus A319
...,...,...
63,BJ 320INR,Airbus A320
64,BJ 320INO,Airbus A320
65,OL 321ABY,Airbus A321
66,X9 320VEB,Airbus A320


In [22]:
# Map aircraft_model to df_train_prepro using aircraft_code
df_train_prepro['aircraft_model'] = df_train_prepro['aircraft_code'].map(
    df_aircraft_model.set_index('aircraft_code')['aircraft_model']
)

In [23]:
df_train_prepro

,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,dep_hour,dep_day,...,dep_precip,dep_wind,arr_temp,arr_precip,arr_wind,Country,City,is_holiday,holiday_length,aircraft_model
0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-19.0,9300.0,4,16,...,9.9,23.6,12.5,0.0,16.5,Tunisia,Tunis,0,0,Airbus A320
1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-48.0,10200.0,14,17,...,0.0,41.2,2.7,2.6,8.8,Tunisia,Djerba,0,0,Boeing 737-600
2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-16.0,15600.0,19,20,...,1.3,6.8,22.3,23.9,11.8,Tunisia,Tunis,0,0,Airbus A320-200
3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-8.0,10200.0,20,21,...,0.0,10.8,10.3,0.0,5.0,Turkey,Istanbul,0,0,Airbus A320-200
4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-37.0,8700.0,17,22,...,0.0,9.8,10.6,2.0,4.6,Morocco,Casablanca,0,0,Airbus A320-200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0,8,18,...,0.0,14.2,17.1,0.0,20.2,Tunisia,Tunis,0,0,Airbus A318
99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0,10,5,...,1.2,10.7,17.3,0.0,12.5,France,Paris,0,0,Boeing 737-600
99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0,9,5,...,0.0,12.4,17.3,0.0,12.5,Belgium,Brussels,0,0,Boeing 737-600
99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0,18,4,...,1.2,8.2,16.7,0.0,19.6,France,Paris,0,0,Airbus A320


In [29]:
df_passenger = pd.read_csv('data/passenger_data.csv')
df_passenger

,IATA_Code,2016 Passengers,2017 Passengers,2018 Passengers
0,AAE,"287,421","302,492","318,116"
1,AAL,"1,518,879","1,607,987","1,643,303"
2,ABJ,"1,860,000","2,080,000","2,200,000"
3,ADB,"11,951,183","12,400,000","13,100,000"
4,AHU,"92,083","122,036","179,355"
...,...,...,...,...
123,VKO,"16,700,000","18,100,000","21,400,000"
124,VNO,"224,157","235,365","247,133"
125,VRN,"2,750,000","3,100,000","3,360,000"
126,YUL,"10,000,000","10,500,000","11,000,000"


In [31]:
# Ensure departure_date is datetime
df_train_prepro['departure_date'] = pd.to_datetime(df_train_prepro['departure_date'])

# Extract year from departure_date
df_train_prepro['year'] = df_train_prepro['departure_date'].dt.year

# Melt df_passenger to long format for easy merging
df_passenger_long = df_passenger.melt(
    id_vars=['IATA_Code'],
    value_vars=['2016 Passengers', '2017 Passengers', '2018 Passengers'],
    var_name='year_col',
    value_name='num_passenger_year'
)
df_passenger_long['year'] = df_passenger_long['year_col'].str.extract(r'(\d{4})').astype(int)

# Merge passenger info into flight data (use 'departure_point' instead of 'IATA_Code')
df_train_prepro = df_train_prepro.merge(
    df_passenger_long[['IATA_Code', 'year', 'num_passenger_year']],
    left_on=['departure_point', 'year'],
    right_on=['IATA_Code', 'year'],
    how='left'
)

# Drop the temporary columns if you don't need them
df_train_prepro = df_train_prepro.drop(columns=['year', 'IATA_Code'])

In [32]:
df_train_prepro

,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,dep_hour,dep_day,...,dep_wind,arr_temp,arr_precip,arr_wind,Country,City,is_holiday,holiday_length,aircraft_model,num_passenger_year
0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-19.0,9300.0,4,16,...,23.6,12.5,0.0,16.5,Tunisia,Tunis,0,0,Airbus A320,"5,200,000"
1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-48.0,10200.0,14,17,...,41.2,2.7,2.6,8.8,Tunisia,Djerba,0,0,Boeing 737-600,"993,429"
2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-16.0,15600.0,19,20,...,6.8,22.3,23.9,11.8,Tunisia,Tunis,0,0,Airbus A320-200,"5,200,000"
3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-8.0,10200.0,20,21,...,10.8,10.3,0.0,5.0,Turkey,Istanbul,0,0,Airbus A320-200,"60,378,393"
4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-37.0,8700.0,17,22,...,9.8,10.6,2.0,4.6,Morocco,Casablanca,0,0,Airbus A320-200,"8,244,481"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0,8,18,...,14.2,17.1,0.0,20.2,Tunisia,Tunis,0,0,Airbus A318,"6,000,000"
99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0,10,5,...,10.7,17.3,0.0,12.5,France,Paris,0,0,Boeing 737-600,"33,120,685"
99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0,9,5,...,12.4,17.3,0.0,12.5,Belgium,Brussels,0,0,Boeing 737-600,"25,700,000"
99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0,18,4,...,8.2,16.7,0.0,19.6,France,Paris,0,0,Airbus A320,"33,120,685"


In [34]:
# Clean and convert 'num_passenger_year' to integer
df_train_prepro['num_passenger_year'] = (
    df_train_prepro['num_passenger_year']
    .replace(',', '', regex=True)
    .astype(float)
    .astype('Int64')
)

In [35]:
df_train_prepro

,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,dep_hour,dep_day,...,dep_wind,arr_temp,arr_precip,arr_wind,Country,City,is_holiday,holiday_length,aircraft_model,num_passenger_year
0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-19.0,9300.0,4,16,...,23.6,12.5,0.0,16.5,Tunisia,Tunis,0,0,Airbus A320,5200000
1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-48.0,10200.0,14,17,...,41.2,2.7,2.6,8.8,Tunisia,Djerba,0,0,Boeing 737-600,993429
2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-16.0,15600.0,19,20,...,6.8,22.3,23.9,11.8,Tunisia,Tunis,0,0,Airbus A320-200,5200000
3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-8.0,10200.0,20,21,...,10.8,10.3,0.0,5.0,Turkey,Istanbul,0,0,Airbus A320-200,60378393
4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-37.0,8700.0,17,22,...,9.8,10.6,2.0,4.6,Morocco,Casablanca,0,0,Airbus A320-200,8244481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0,8,18,...,14.2,17.1,0.0,20.2,Tunisia,Tunis,0,0,Airbus A318,6000000
99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0,10,5,...,10.7,17.3,0.0,12.5,France,Paris,0,0,Boeing 737-600,33120685
99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0,9,5,...,12.4,17.3,0.0,12.5,Belgium,Brussels,0,0,Boeing 737-600,25700000
99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0,18,4,...,8.2,16.7,0.0,19.6,France,Paris,0,0,Airbus A320,33120685


In [36]:
df_train_prepro.to_csv('data/data_added_aircraft_passengers.csv', index=False)